# fuzzy string matching
## 📚 resources
- `seatgeek` / [`thefuzz`](https://github.com/seatgeek/thefuzz) > README > [Usage](https://github.com/seatgeek/thefuzz/blob/83bea3d4a109a5d0c2e07334b504953cda4959c5/README.rst#Usage)
## details
- **to select, from one or more possible labels, the label that best matches the IRI slug**  
- *NOTE 🚩 that this is only enabled by the fact that ONS IRI slugs are created from labels with space, punctuation, etc. stripped*  


In [57]:
from some_ons import some_ons
from all_ons import all_ons
# see below

In [58]:
from thefuzz import fuzz, process
from rdflib import Graph, RDFS

In [60]:
with open("fuzzy_matching.md", 'w+') as mdfile:
    for entry in some_ons: # see above first cell imports
        mdfile.write(f"# {entry[0]}\n")
        iri_slug = entry[0].split('/')[-1]
        try:
            data = Graph().parse(f"{entry[0]}.nt")
            for s, p, o in data.triples((None, RDFS.label, None)):
                label = o
            mdfile.write(f"## rdfs:label = '{label}'\n")
        except:
            mdfile.write(f"## [!] Resource with IRI {entry[0]} does not seem to exist in ONS\n")
        mdfile.write(f"## Fuzzy matching iri_slug '{iri_slug}' to Creator Display values:\n")
        for item in entry[1::]:
            mdfile.write(f"### {item}\n")
            mdfile.write(f"- fuzz.ratio = {fuzz.ratio(iri_slug, item)}\n")
            mdfile.write(f"- fuzz.partial_ratio = {fuzz.partial_ratio(iri_slug, item)}\n")
            mdfile.write(f"- fuzz.token_sort_ration = {fuzz.token_sort_ratio(iri_slug, item)}\n")
            # to do -- remove parenthisized role information from creator display values prior to comparing
